In [1]:
import numpy as np
import pandas as pd
import spacy
import nltk
from spacy.tokens import DocBin
from tqdm import tqdm

In [2]:
data = pd.read_json('..\\data\\Corona2.json')
data.head()

,examples
0,"{'id': '18c2f619-f102-452f-ab81-d26f7e283ffe',..."
1,"{'id': '487c93e3-0d45-4088-a378-cf3a01c8953d',..."
2,"{'id': 'd5056874-895a-4a7f-9e0f-828d414d65d9',..."
3,"{'id': '20c792c7-0c4b-42d0-8127-0e04113db384',..."
4,"{'id': 'f5359e0d-4d4a-4707-95a3-4c627fc4a83b',..."


In [3]:
data.keys()

Index(['examples'], dtype='object')

In [4]:
data["examples"][0].keys()

dict_keys(['id', 'content', 'metadata', 'annotations', 'classifications'])

In [5]:
data["examples"][0]["id"]

'18c2f619-f102-452f-ab81-d26f7e283ffe'

In [6]:
data["examples"][0]["content"]

"While bismuth compounds (Pepto-Bismol) decreased the number of bowel movements in those with travelers' diarrhea, they do not decrease the length of illness.[91] Anti-motility agents like loperamide are also effective at reducing the number of stools but not the duration of disease.[8] These agents should be used only if bloody diarrhea is not present.[92]\n\nDiosmectite, a natural aluminomagnesium silicate clay, is effective in alleviating symptoms of acute diarrhea in children,[93] and also has some effects in chronic functional diarrhea, radiation-induced diarrhea, and chemotherapy-induced diarrhea.[45] Another absorbent agent used for the treatment of mild diarrhea is kaopectate.\n\nRacecadotril an antisecretory medication may be used to treat diarrhea in children and adults.[86] It has better tolerability than loperamide, as it causes less constipation and flatulence.[94]"

In [7]:
data["examples"][0]["metadata"]

{}

In [8]:
data["examples"][0]["annotations"]

[{'id': '0825a1bf-6a6e-4fa2-be77-8d104701eaed',
  'tag_id': 'c06bd022-6ded-44a5-8d90-f17685bb85a1',
  'end': 371,
  'start': 360,
  'example_id': '18c2f619-f102-452f-ab81-d26f7e283ffe',
  'tag_name': 'Medicine',
  'value': 'Diosmectite',
  'correct': None,
  'human_annotations': [{'timestamp': '2020-03-21T00:24:32.098000Z',
    'annotator_id': 1,
    'tagged_token_id': '0825a1bf-6a6e-4fa2-be77-8d104701eaed',
    'name': 'Ashpat123',
    'reason': 'exploration'}],
  'model_annotations': []},
 {'id': '145f62b1-bbf5-42f1-8ad5-9c7e08337bf0',
  'tag_id': 'c06bd022-6ded-44a5-8d90-f17685bb85a1',
  'end': 408,
  'start': 383,
  'example_id': '18c2f619-f102-452f-ab81-d26f7e283ffe',
  'tag_name': 'Medicine',
  'value': 'aluminomagnesium silicate',
  'correct': None,
  'human_annotations': [{'timestamp': '2020-03-21T00:24:43.692000Z',
    'annotator_id': 1,
    'tagged_token_id': '145f62b1-bbf5-42f1-8ad5-9c7e08337bf0',
    'name': 'Ashpat123',
    'reason': 'exploration'}],
  'model_annotations':

In [9]:
data["examples"][0]["classifications"]

[]

In [10]:
data["examples"][0]["annotations"][0].keys()

dict_keys(['id', 'tag_id', 'end', 'start', 'example_id', 'tag_name', 'value', 'correct', 'human_annotations', 'model_annotations'])

In [11]:
training_data = [{'text': example["content"], "entities": [(annotation["start"], annotation["end"], annotation["tag_name"].upper()) 
                                           for annotation in example["annotations"]]} 
 for example in data["examples"]]

In [12]:
training_data[0].keys()

dict_keys(['text', 'entities'])

In [13]:
training_data[0]["text"]

"While bismuth compounds (Pepto-Bismol) decreased the number of bowel movements in those with travelers' diarrhea, they do not decrease the length of illness.[91] Anti-motility agents like loperamide are also effective at reducing the number of stools but not the duration of disease.[8] These agents should be used only if bloody diarrhea is not present.[92]\n\nDiosmectite, a natural aluminomagnesium silicate clay, is effective in alleviating symptoms of acute diarrhea in children,[93] and also has some effects in chronic functional diarrhea, radiation-induced diarrhea, and chemotherapy-induced diarrhea.[45] Another absorbent agent used for the treatment of mild diarrhea is kaopectate.\n\nRacecadotril an antisecretory medication may be used to treat diarrhea in children and adults.[86] It has better tolerability than loperamide, as it causes less constipation and flatulence.[94]"

In [14]:
training_data[0]["entities"]

[(360, 371, 'MEDICINE'),
 (383, 408, 'MEDICINE'),
 (104, 112, 'MEDICALCONDITION'),
 (679, 689, 'MEDICINE'),
 (6, 23, 'MEDICINE'),
 (25, 37, 'MEDICINE'),
 (461, 470, 'MEDICALCONDITION'),
 (577, 589, 'MEDICINE'),
 (853, 865, 'MEDICALCONDITION'),
 (188, 198, 'MEDICINE'),
 (754, 762, 'MEDICALCONDITION'),
 (870, 880, 'MEDICALCONDITION'),
 (823, 833, 'MEDICINE'),
 (852, 853, 'MEDICALCONDITION'),
 (461, 469, 'MEDICALCONDITION'),
 (535, 543, 'MEDICALCONDITION'),
 (692, 704, 'MEDICINE'),
 (563, 571, 'MEDICALCONDITION')]

In [15]:
#Processing data using spacy
nlp = spacy.blank("en")
doc_bin = DocBin()

In [16]:
from spacy.util import filter_spans

for training_example in tqdm(training_data):
    text = training_example["text"]
    labels = training_example["entities"]
    doc = nlp.make_doc(text)
    ents = []
    for start, end, label in labels:
        span = doc.char_span(start, end, label = label, alignment_mode = "contract")
        if span is None:
            print("skipping entity")
        else:
            ents.append(span)
    filtered_ents = filter_spans(ents)
    doc.ents = filtered_ents
    doc_bin.add(doc)
doc_bin.to_disk("train.spacy")

100%|█████████████████████████████████████████████████████████████████████████████████| 31/31 [00:00<00:00, 757.96it/s]

skipping entity
skipping entity
skipping entity
skipping entity
skipping entity
skipping entity
skipping entity
skipping entity
skipping entity
skipping entity
skipping entity
skipping entity
skipping entity
skipping entity
skipping entity
skipping entity


In [17]:
!python -m spacy init fill-config base_config.cfg config.cfg

[+] Auto-filled config with all values
[+] Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [18]:
!python -m spacy train config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./train.spacy

[+] Created output directory: output
[i] Saving to output directory: output
[i] Using CPU

=========================== Initializing pipeline ===========================
[+] Initialized pipeline

============================= Training pipeline =============================
[i] Pipeline: ['tok2vec', 'ner']
[i] Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00    153.29    0.00    0.00    0.00    0.00
  7     200       2103.31   3325.40   79.76   79.61   79.92    0.80
 14     400        196.12    638.35   96.47   96.09   96.85    0.96
 22     600        227.61    238.22   97.65   97.27   98.03    0.98
 30     800        220.65    224.09   98.43   98.43   98.43    0.98
 40    1000        283.43    177.79   98.02   98.41   97.64    0.98
 51    1200        205.13    184.89   98.42   98.81   98.03    0.98
 65    1400        262.04    177.62   98.43   98.05 

In [20]:
nlp_ner = spacy.load("output/model-best")

In [27]:
doc =nlp_ner("""The patient was diagnosed with diabetes mellitus and prescribed insulin. He complained of severe chest pain and shortness of breath. 
Amoxicillin was administered to treat a bacterial infection. He underwent angioplasty and two stents placed a year ago. His BP and cardiac enzyme test were normal.
He is on aspirin and plavix. The patient also went under polypectomy. Plant is to start him on Bevacizumab and radiation therapy. The patient is also suffering
from Xeloda and hepatitis
""")

In [28]:
spacy.displacy.render(doc, style = "ent", jupyter = True)